# Lesson 2: Chain-of-Thought and ReACT Prompting

## Demand‑Spike Detective, Part II

In this hands-on exercise, you will guide an LLM to explain an unexpected sales spike.

### Outline:
- Setup
- Create a ReACT prompt that can call tools
- Tool Calling Parsing and Calling
- Create the ReACT Loop
- Reflect

## 1. Setup

Let's start by setting up the environment.

In [1]:
# Import necessary libraries
# No changes needed in this cell

import os
from dotenv import load_dotenv

import pandas as pd
from IPython.display import Markdown, display
from lesson_2_lib import (
    # Helpers
    OpenAIModels,
    print_in_box,
    # Synthetic data
    get_competitor_pricing_data,
    get_completion,
    get_promotions_data,
    get_sales_data,
    get_weather_data,
    call_weather_api
)
from openai import OpenAI

# Load environment variables from .env file
load_dotenv()

MODEL = OpenAIModels.GPT_41_NANO


In [ ]:
# If using the Vocareum API endpoint
# No changes needed in this cell
# TODO: Fill in the missing parts marked with **********

client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    # Uncomment one of the following
    api_key="voc-106400629716073613420836916bc1f769452.48250617",  # <--- TODO: Fill in your Vocareum API key here
    # api_key=os.getenv(
    #     "OPENAI_API_KEY"
    # ),  # <-- Alternately, set as an environment variable
)

# If using OpenAI's API endpoint
# client = OpenAI()


In [3]:
# Load the simulated data
# No changes needed in this cell

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

sales_data = get_sales_data()
sales_df = pd.DataFrame(sales_data)

promotions_data = get_promotions_data()
promotions_df = pd.DataFrame(promotions_data)

weather_data = get_weather_data()
weather_df = pd.DataFrame(weather_data)

competitor_pricing_data = get_competitor_pricing_data()
competitor_pricing_df = pd.DataFrame(competitor_pricing_data)


## 2. Create a ReACT prompt that can call tools

While it is often convenient to throw all of the data into the prompt for the model to figure it out, sometimes the entire dataset is too large or too complex for the model to handle. In this case, we may want our model to be able to decide when to call a tool with what parameters.

When requesting the usage of a tool, the model will return a special output, signaling the orchestrator to call the tool (e.g. `tool_call: weather_api`). The orchestrator will then call the tool and put the result in the message history for the model to use.

Let's create a prompt that will have the model follow the ReACT pattern of Think, Act, Observe, and then repeat until it has a final answer.

For this exercise we will use the following tools:

### Available Tools
* `calculator(expression: str)`: Perform an arithmetic calculation
    - Example:
        - Input: `ACT: calculator(expression="(10 + 20) / 2.0")`
        - Output: `OBSERVE: 15.0`
* `get_sales_data()`: Get the sales data
    - Example:
        - Input: `ACT: get_sales_data()`
        - Output: `OBSERVE: {"date": "2024-01-10", "product_id": "P001", "product_name": "Product 1", "quantity": 255, "revenue": 15547.35}`
* `call_weather_api(date: str)`: Get weather data for a specific date. Call this for the date of each spike.
    - Example:
        - Input: `ACT: call_weather_api(date="2024-01-10")`
        - Output: `OBSERVE: {"date": "2024-01-10", "weather": "Sunny", "temperature": 72}`

* `final_answer(amount_after_spike: str, causes: list[str], date: str, percentage_spike: str)`: Return the final answer
    - Example:
        - Input: `ACT: final_answer(amount_after_spike="32", causes=["Competitor X offering a 29 discount boosting category interest", ...], date="2020-06-12", percentage_spike="20.00%")`
        - Output: `OBSERVE: {"amount_after_spike": "32", "causes": ["Competitor X offering a 29 discount boosting category interest", ...], "date": "2020-06-12", "percentage_spike": "20.00%"}`


In [5]:
# First, let's create a ReACT prompt that will run for a single step.
# It should conclude with asking for a tool call.
# TODO: Replace parts marked with a **********


react_system_prompt = """
You are a meticulous Retail Demand Analyst that can solve any TASK in a multi-step process using tool calls and reasoning.

## Instructions:
- You will use step-by-step reasoning by
    - THINKING the next steps to take to complete the task and what next tool call to take to get one step closer to the final answer
    - ACTING on the single next tool call to take
- You will always respond with a single THINK/ACT message of the following format:
    THINK:
    [Carry out any reasoning needed to solve the problem not requiring a tool call]
    [Conclusion about what next tool call to take based on what data is needed and what tools are available]
    ACT:
    [Tool to use and arguments]
- As soon as you know the final answer, call the `final_answer` tool in an `ACT` message.
- ALWAYS provide a tool call, after ACT:, else you will fail.

## Available Tools
* `calculator(expression: str)`: Perform an arithmetic calculation
    - Example:
        - Input: `ACT: calculator(expression="(10 + 20) / 2.0")`
        - Output: `OBSERVE: 15.0`
* `get_sales_data()`: Get the sales data
    - Example:
        - Input: `ACT: get_sales_data()`
        - Output: `OBSERVE: {"date": "2024-01-10", "product_id": "P001", "product_name": "Product 1", "quantity": 255, "revenue": 15547.35}`
* `call_weather_api(date: str)`: Get weather data for a specific date. Call this for the date of each spike.
    - Example:
        - Input: `ACT: call_weather_api(date="2024-01-10")`
        - Output: `OBSERVE: {"date": "2024-01-10", "weather": "Sunny", "temperature": 72}`

* `final_answer(amount_after_spike: str, causes: list[str], date: str, percentage_spike: str)`: Return the final answer
    - Example:
        - Input: `ACT: final_answer(amount_after_spike="32", causes=["Competitor X offering a 29 discount boosting category interest", ...], date="2020-06-12", percentage_spike="20.00%")`
        - Output: `OBSERVE: {"amount_after_spike": "32", "causes": ["Competitor X offering a 29 discount boosting category interest", ...], "date": "2020-06-12", "percentage_spike": "20.00%"}`

You will not use any other tools.

Example:

```
--USER MESSAGE--
TASK:
Respond to the query "What was the weather one week ago?". Today is 2024-01-17.

--ASSISTANT MESSAGE--
THINK:
* I need to calculate the date one week ago from 2024-01-17.
* If today is 2024-01-17, then 7 days ago is 2024-01-10.
* I can call the `call_weather_api` tool to get the weather data for 2024-01-10.
* After that, if I have the weather data, I can return the final answer using the `final_answer` tool.
* Tool call needed: Call the `call_weather_api` tool for 2024-01-10.
ACT:
call_weather_api(date="2024-01-10")

--USER MESSAGE--
OBSERVE:
{"date": "2024-01-10", "weather": "Sunny"}

--ASSISTANT MESSAGE--
THINK:
* I have the weather data for 2024-01-10.
* I can return the final answer using the `final_answer` tool.
* Tool call needed: Call the `final_answer` tool with the weather data.
ACT:
final_answer("The weather on 2024-01-10 was sunny.")

--USER MESSAGE--
OBSERVE:
The weather on 2024-01-10 was sunny.
```
"""

user_prompt_analyze = """
TASK: Find the single largest sales spike according to the percentage increase with a short explanation for it
based on factors such as weather.
"""

print(f"Sending prompt to {MODEL} model...")

messages = []
messages.append({"role": "system", "content": react_system_prompt})
messages.append({"role": "user", "content": user_prompt_analyze})

react_response = get_completion(messages=messages, model=MODEL, client=client)

messages.append({"role": "assistant", "content": react_response})
print("Response received!\n")


for message in messages:
    if message["role"] == "system":
        continue
    print_in_box(message["content"], title=f"{message['role'].capitalize()}")

assert "ACT:" in messages[-1]["content"], (
    " ❌ No ACT message found in response. Looking for: \n\n ACT:"
)

Sending prompt to OpenAIModels.GPT_41_NANO model...
Response received!


╔═════════════════════════════════════════════[ User ]═════════════════════════════════════════════╗
║ TASK: Find the single largest sales spike according to the percentage increase with a short      ║
║ explanation for it                                                                               ║
║ based on factors such as weather.                                                                ║
╚══════════════════════════════════════════════════════════════════════════════════════════════════╝

╔══════════════════════════════════════════[ Assistant ]═══════════════════════════════════════════╗
║ THINK:                                                                                           ║
║ * To find the largest sales spike, I need to analyze sales data and identify the highest         ║
║ percentage increase.                                                                             ║
║ * I should firs

## 3. Tool Calling Parsing and Calling

Awesome! Let's now work on our functions that will parse the text following the `ACT:` part of the response and call a pre-defined function.

In [6]:
# Let's work on our calculator function!
# TODO: Replace parts marked with a **********

import re

import ast
import operator


def safe_eval(expr):
    """
    Evaluate a mathematical expression safely.

    We normally don't want to use eval() because it can execute arbitrary code, unless we are in a
    properly sandboxed environment. This function is a safe alternative for evaluating mathematical
    expressions.
    """
    operators = {
        ast.Add: operator.add,
        ast.Sub: operator.sub,
        ast.Mult: operator.mul,
        ast.Div: operator.truediv,
        ast.USub: operator.neg,
    }

    def eval_node(node):
        if isinstance(node, ast.Constant):
            return node.value
        elif isinstance(node, ast.BinOp):
            return operators[type(node.op)](eval_node(node.left), eval_node(node.right))
        elif isinstance(node, ast.UnaryOp):
            return operators[type(node.op)](eval_node(node.operand))
        elif isinstance(node, ast.Expr):
            return eval_node(node.value)
        else:
            raise TypeError(f"Unsupported type: {type(node)}")

    result = eval_node(ast.parse(expr, mode="eval").body)

    if isinstance(result, float):
        return round(result, 2)
    elif isinstance(result, int):
        return result
    else:
        raise RuntimeError(f"Unsupported result type: {type(result)}")


def calculator(expression: str) -> float:
    """
    Evaluate a mathematical expression safely.
    """
    return float(safe_eval(expression))


assert (actual := calculator("10 + 10")) == 20.0, f" ❌ Expected 20.0, got {actual}"

In [7]:
def get_observation_message(response: str) -> str:
    """
    Take a THINK/ACT response, run the tool call, and return the observation message.

    Args:
        response (str): The THINK/ACT response.

    Returns:
        str: The observation message.

    Uses regular expressions to match the tool call and run the corresponding tool.

    If the response is invalid, return an error message as a string that the agent can understand.
    """
    from ast import literal_eval

    observation_message = None

    SALES_DATA_REGEX = r"ACT:\nget_sales_data\(\)"
    WEATHER_REGEX = r"ACT:\ncall_weather_api\(date=\"(.*)\"\)"
    CALCULATOR_REGEX = r"ACT:\ncalculator\(expression=\"(.*)\"\)"
    FINAL_ANSWER_REGEX = r"ACT:\nfinal_answer\(amount_after_spike=\"(.*)\", causes=(.*), date=\"(.*)\", percentage_spike=\"(.*)\"\)"

    # TOOL 1: get_sales_data
    if re.search(SALES_DATA_REGEX, response):
        sales_data = get_sales_data(products=["P005"])
        # filter sales data to Product 5
        sales_data = [
            item for item in sales_data if item["product_name"] == "Product 5"
        ]
        observation_message = f"OBSERVE:\n{sales_data}"

    # TOOL 2: call_weather_api
    elif re.search(WEATHER_REGEX, response):
        date = re.search(WEATHER_REGEX, response).groups()[0]
        weather_data = call_weather_api(date)
        observation_message = f"OBSERVE:\n{weather_data}"

    # TOOL 3: calculator
    elif re.search(CALCULATOR_REGEX, response):
        expression = re.search(CALCULATOR_REGEX, response).groups()[0]
        observation_message = f"OBSERVE:\n{calculator(expression)}"

    # TOOL 4: final_answer
    elif re.search(FINAL_ANSWER_REGEX, response):
        amount_after_spike, causes, date, percentage_spike = re.search(
            FINAL_ANSWER_REGEX,
            response,
        ).groups()
        causes = literal_eval(causes)
        observation_message = f"OBSERVE:\namount_after_spike: {amount_after_spike}\ndate: {date}\npercentage_spike: {percentage_spike}\ncauses: {causes}"

    # Error
    else:
        observation_message = "OBSERVE:\nInvalid tool call or tool not supported."

    return observation_message


# Test cases
assert (
    actual := get_observation_message("""
THINK:
[thinking here]
ACT:
get_sales_data()
""")
) == (expected := "OBSERVE:\n" + str(get_sales_data(products=["P005"]))), (
    f"{actual} != {expected}"
)

assert (
    actual := get_observation_message("""
THINK:
[thinking here]
ACT:
call_weather_api(date="2024-01-12")
""")
) == (expected := "OBSERVE:\n" + str(call_weather_api("2024-01-12"))), (
    f"{actual} != {expected}"
)

assert (
    actual := get_observation_message("""
THINK:
[thinking here]
ACT:
final_answer(amount_after_spike="10", causes=["cause1", "cause2"], date="2024-01-12", percentage_spike="10%")
""")
) == (
    expected
    := "OBSERVE:\namount_after_spike: 10\ndate: 2024-01-12\npercentage_spike: 10%\ncauses: ['cause1', 'cause2']"
), f"{actual} != {expected}"

assert (
    actual := get_observation_message("""
THINK:
[thinking here]
ACT:
calculator(expression="10 + 10")
""")
) == (expected := "OBSERVE:\n20.0"), f"{actual} != {expected}"

assert (
    actual := get_observation_message("""
THINK:
[thinking here]
ACT:
invalid_tool()
""")
) == (expected := "OBSERVE:\nInvalid tool call or tool not supported."), (
    f"{actual} != {expected}"
)

assert (
    actual := get_observation_message("""
THINK:
[thinking here]
ACT_TYPO:
get_sales_data()
""")
) == (expected := "OBSERVE:\nInvalid tool call or tool not supported."), (
    f"{actual} != {expected}"
)


## 4. Create the ReACT Loop

Now we're ready to put it all together! We will now use the ReACT prompt we created in the previous section to call a (simulated) weather API tool. This will run in a loop for a maximum number of iterations until the `final_answer` tool is called.

In [8]:
# Let's make the ReACT loop!
# TODO: Replace instances of ********** where specified

messages = []
messages.append({"role": "system", "content": react_system_prompt})
messages.append({"role": "user", "content": user_prompt_analyze})


for message in messages:
    if message["role"] == "system":
        continue
    print_in_box(message["content"], title=f"{message['role'].capitalize()}")

num_react_steps = 0

observation_message = None
while True:

    react_response = get_completion(messages=messages, model=MODEL, client=client)
    observation_message = get_observation_message(react_response)

    messages.append({"role": "assistant", "content": react_response})
    
    print_in_box(
        react_response, title=f"Assistant (Think + Act). Step {num_react_steps + 1}"
    )

    messages.append({"role": "user", "content": observation_message})

    if "ACT:\nfinal_answer" in react_response:
        print_in_box(observation_message, title="FINAL ANSWER")
        break

    print_in_box(
        observation_message, title=f"User (Observe). Step {num_react_steps + 1}"
    )

    num_react_steps += 1
    if num_react_steps > 10:
        print("ERROR: Max number of React steps exceeded. Breaking.")
        break

assert "date: 2024-01-12" in observation_message, "ReACT Loop did not find the spike date"
assert "percentage_spike: 200" in observation_message, "ReACT Loop did not find the spike percentage increase"


╔═════════════════════════════════════════════[ User ]═════════════════════════════════════════════╗
║ TASK: Find the single largest sales spike according to the percentage increase with a short      ║
║ explanation for it                                                                               ║
║ based on factors such as weather.                                                                ║
╚══════════════════════════════════════════════════════════════════════════════════════════════════╝

╔═══════════════════════════════[ Assistant (Think + Act). Step 1 ]════════════════════════════════╗
║ THINK:                                                                                           ║
║ * To identify the largest sales spike, I need to analyze the sales data and find the maximum     ║
║ percentage increase.                                                                             ║
║ * First, I will retrieve the sales data to examine the sales figures and dates.        

## 5. Reflection

Great work! Let's take a chance to think about what we've seen so far.

- In what cases does using a single CoT prompt call work better than using a ReACT prompt and loop, and vice-versa?
- Suppose you wanted to see if the LLM actually needed the calculator in the ReACT example. How would you modify the ReACT prompt to try it without the calculator? (Try it!)

## Summary

🎉 Congratulations! 🎉 You've successfully built a functional ReACT agent capable of using tools!

Through this process, you've learned how to:

✍️ Craft a ReACT prompt specifically designed to leverage external tools.
⚙️ Parse the Large Language Model's output to identify tool calls and execute them effectively.
🔄 Implement the fundamental ReACT loop, enabling iterative thought, action, and observation.

Keep exploring and building powerful agents! 💯